We first create one netcdf files covering 2003 to 2019

In [1]:
import os
import pandas as pd
import geopandas as gpd
import xarray as xr


os.chdir('/Users/babak.jfard/projects/NASA_IMERG_Respiratory')

In [2]:
# Get a list of files and reduce it to years 2003 to 2019
files = os.listdir('Data/Monthly')

start_year = 2003
end_year = 2019

files = [f for f in files if int(f.split('.')[3][0:4]) in range(start_year, end_year+1)]
files.sort()


In [4]:
test = xr.open_dataset('Data/Monthly/'+files[len(files)-30])

In [8]:
test

<xarray.Dataset> Size: 80MB
Dimensions:  (lon: 6700, lat: 3000)
Coordinates:
  * lon      (lon) float64 54kB -132.0 -132.0 -132.0 ... -65.03 -65.01 -65.0
  * lat      (lat) float64 24kB 25.0 25.01 25.02 25.03 ... 54.97 54.99 54.99
    time     int64 8B 2003
Data variables:
    GWRPM25  (lat, lon) float32 80MB ...
Attributes:
    Title:            Hybrid PM_2_._5 [\mug/m^3]
    Contact:          Aaron van Donkelaar <aaron.vandonkelaar@wustl.edu>
    References:       van Donkelaar et al., Monthly Global Estimates of Fine ...
    Filename:         C:\Users\Aaron\Documents\MATLAB\PM25_wMelanie\GlobalMon...
    Format:           NetCDF-4
    Delta_Lat:        0.010000228881835938
    Delta_Lon:        0.0099945068359375
    SpatialCoverage:  NorthAmerica
    Start_Date:       20170101
    End_Date:         20171231

In [11]:
test.dims

FrozenMappingWarningOnValuesAccess({'lon': 6700, 'lat': 3000})

In [7]:
test =test.assign_coords(time=start_year)

In [13]:
#import xarray as xr
#import pandas as pd

# List to store datasets
datasets = []
file_path = 'Data/Monthly/'

# Iterate over .nc files
for file in files:
    try:
        # Open the dataset
        ds = xr.open_dataset(file_path+file)
        
        # Check if time dimension already exists
        if 'time' not in ds.dims:
            # If time dimension doesn't exist, create it
            YM = file.split('.')[3][0:6]+'01'
            start_date = pd.to_datetime(YM)
            
            ds = ds.assign_coords(time=start_date)
        
        # Append dataset to list
        datasets.append(ds)
        
    except Exception as e:
        print(f"Error processing file {file}: {e}")

# Combine datasets along 'time' dimension
combined_ds = xr.concat(datasets, dim='time')




In [18]:
combined_ds.attrs

{'Title': 'Hybrid PM_2_._5 [\\mug/m^3]',
 'Contact': 'Aaron van Donkelaar <aaron.vandonkelaar@wustl.edu>',
 'References': 'van Donkelaar et al., Monthly Global Estimates of Fine Particulate Matter and Their Uncertainty, Environ. Sci. & Tech., doi:10.1021/acs.est.1c05309.',
 'Filename': 'C:\\Users\\Aaron\\Documents\\MATLAB\\PM25_wMelanie\\GlobalMonthlyGWR-Geo20230818-GM20230905-V20230921\\NetCDF\\GWRPM25\\NorthAmerica\\Monthly\\V5GL04.HybridPM25.NorthAmerica.200301-200301.nc',
 'Format': 'NetCDF-4',
 'Delta_Lat': 0.010000228881835938,
 'Delta_Lon': 0.0099945068359375,
 'SpatialCoverage': 'NorthAmerica',
 'Start_Date': '20030101',
 'End_Date': '20191231'}

In [17]:
combined_ds.attrs['End_Date']='20191231'

In [19]:
combined_ds

<xarray.Dataset> Size: 16GB
Dimensions:  (lon: 6700, lat: 3000, time: 204)
Coordinates:
  * lon      (lon) float64 54kB -132.0 -132.0 -132.0 ... -65.03 -65.01 -65.0
  * lat      (lat) float64 24kB 25.0 25.01 25.02 25.03 ... 54.97 54.99 54.99
  * time     (time) datetime64[ns] 2kB 2003-01-01 2003-02-01 ... 2019-12-01
Data variables:
    GWRPM25  (time, lat, lon) float32 16GB nan nan nan nan ... 1.1 1.1 1.1 1.1
Attributes:
    Title:            Hybrid PM_2_._5 [\mug/m^3]
    Contact:          Aaron van Donkelaar <aaron.vandonkelaar@wustl.edu>
    References:       van Donkelaar et al., Monthly Global Estimates of Fine ...
    Filename:         C:\Users\Aaron\Documents\MATLAB\PM25_wMelanie\GlobalMon...
    Format:           NetCDF-4
    Delta_Lat:        0.010000228881835938
    Delta_Lon:        0.0099945068359375
    SpatialCoverage:  NorthAmerica
    Start_Date:       20030101
    End_Date:         20191231

In [20]:
# Saving 
combined_ds.to_netcdf('Data/V5GL04.HybridPM25.NorthAmerica.200301-201912.nc')